<a href="https://www.kaggle.com/code/krisnasatya/training-model-llama3-1-8b-instruct-v2?scriptVersionId=268751690" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6qe601ga/unsloth_15c35020cfa44fada6640244cff52c06
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6qe601ga/unsloth_15c35020cfa44fada6640244cff52c06
  Resolved https://github.com/unslothai/unsloth.git to commit 85923d098dec9dce4b698f33e997d59144d6250d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24

In [2]:
import torch
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()

gc.collect()

print("Cache memori GPU sudah dibersihkan.")

Cache memori GPU sudah dibersihkan.


## Local Inference on GPU 
Model page: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)

print("Login ke Hugging Face berhasil!")

2025-10-17 08:53:45.734765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760691225.911229      77 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760691225.957285      77 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Login ke Hugging Face berhasil!


In [4]:

dataset_path = "/kaggle/input/personality/alya_10k_unique.json"
dataset = load_dataset("json", data_files=dataset_path, split="train")

print("Contoh data dari dataset:")
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Contoh data dari dataset:
{'id': 13000, 'core_trait': 'Tsundere (Menegur disiplin waktu)', 'context': 'Mahasiswa terlambat mengumpulkan tugas proposal.', 'original_thought_russian': 'Почему снова так? Я просто хочу, чтобы всё было лучше. (Kenapa terlambat mengumpulkan tugas proposal? Waktu itu berharga, jangan disia-siakan.)', 'adapted_dialogue_for_assistant': 'Hah? terlambat mengumpulkan tugas proposal? Waktu itu berharga, jangan disia-siakan. Aku bilang begini karena peduli.'}


In [5]:
from unsloth import FastLanguageModel
import torch

base_model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name,
    max_seq_length = 2048, 
    dtype = None, 
    load_in_4bit = True, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

print("✅ Model dan Tokenizer Unsloth berhasil dimuat!")

/tmp/ipykernel_77/1699199926.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xfo

model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Model dan Tokenizer Unsloth berhasil dimuat!


In [6]:
max_length = 256

tokenized_dataset = dataset.map(
    lambda examples: tokenizer(examples["adapted_dialogue_for_assistant"], truncation=True, max_length=max_length),
    batched=True,
    remove_columns=dataset.column_names
)

print("Dataset berhasil di-tokenisasi.")

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset berhasil di-tokenisasi.


In [7]:
import os, gc, math, random, warnings
import torch
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import FastLanguageModel

warnings.filterwarnings("ignore")
SEED = 3407
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

def _hard_vram_reset():
    for k in list(globals().keys()):
        if k in ("model","tokenizer","trainer"):
            try: del globals()[k]
            except: pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        try: torch.cuda.ipc_collect()
        except: pass
        torch.cuda.set_device(0)


In [8]:
assert 'dataset' in globals(), "Variabel `dataset` (mentah) harus sudah ada dari sel sebelumnya."
if "adapted_dialogue_for_assistant" not in dataset.column_names:
    raise ValueError("Kolom 'adapted_dialogue_for_assistant' tidak ada.")


In [9]:
df = dataset.to_pandas()[["adapted_dialogue_for_assistant"]].copy()
df["text"] = df["adapted_dialogue_for_assistant"].fillna("").astype(str).str.strip()
df = df[df["text"].str.len() > 5].drop_duplicates(subset=["text"], keep="first")
raw_dedup = Dataset.from_pandas(df[["text"]], preserve_index=False)
split_dataset = raw_dedup.train_test_split(test_size=0.1, seed=SEED)
print("Final sizes (dedup):", len(split_dataset["train"]), len(split_dataset["test"]))

def format_instruction(ex):
    return f"<s>[INST] Berikan respons sebagai asisten akademik bernama Alya. [/INST] {ex['text']}</s>"


Final sizes (dedup): 9000 1000


In [10]:
_hard_vram_reset()
BASE_MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct"
cfg = dict(max_seq_length=256, offload_embedding=True, gpu_util=0.70, r=8,
           target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj","lm_head"])

print(f"Load base {BASE_MODEL_ID} @seq={cfg['max_seq_length']}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=cfg["max_seq_length"],
    dtype=None,
    load_in_4bit=True,
    device_map=None,
    offload_embedding=cfg["offload_embedding"],
    gpu_memory_utilization=cfg["gpu_util"],
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = FastLanguageModel.get_peft_model(
    model,
    r=cfg["r"],
    target_modules=cfg["target_modules"],
    lora_alpha=2 * cfg["r"],
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    max_seq_length=cfg["max_seq_length"],
)
if hasattr(model, "config"):
    model.config.use_cache = False
model.to("cuda:0")


Load base unsloth/Meta-Llama-3.1-8B-Instruct @seq=256...
==((====))==  Unsloth 2025.10.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.10.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training lm_head in mixed precision to save VRAM


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [11]:
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
train_size = len(split_dataset["train"])
eff_batch  = per_device_train_batch_size * gradient_accumulation_steps
steps_per_epoch = math.ceil(train_size / max(1, eff_batch))
num_train_epochs = 2
max_steps = steps_per_epoch * num_train_epochs

args = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    warmup_ratio=0.03,
    max_steps=max_steps,
    logging_steps=200,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="paged_adamw_8bit",
    report_to="none",
    output_dir="outputs_fast",
    eval_strategy="no",
    save_strategy="no",
    seed=SEED,
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    skip_memory_metrics=True,
    max_grad_norm=0.3,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=split_dataset["train"],
    formatting_func=format_instruction,
    max_seq_length=cfg["max_seq_length"],
    args=args,
    packing=True,
)

print(f"train_size={train_size}, eff_batch={eff_batch}, steps/epoch={steps_per_epoch}, max_steps={max_steps}")


Generating train split: 0 examples [00:00, ? examples/s]

train_size=9000, eff_batch=16, steps/epoch=563, max_steps=1126


In [12]:
'print("Training...")
trainer.train()
print("Done.")

new_model_name = "Llama-3.1-8B-Instruct-Alya-Unsloth"
trainer.model.save_pretrained(new_model_name, safe_serialization=True)
tokenizer.save_pretrained(new_model_name)
print("Saved to:", new_model_name)

import pandas as pd
import matplotlib.pyplot as plt
logs = pd.DataFrame(trainer.state.log_history)
if 'eval_loss' in logs.columns:
    logs[['loss','eval_loss']].plot()
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Training vs. Evaluation Loss")
    plt.show()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,934 | Num Epochs = 19 | Total steps = 1,126
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 546,308,096 of 8,576,569,344 (6.37% trained)


Training...
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
200,1.206500
400,0.417800
600,0.309200
800,0.278500
1000,0.266800


Done.
Saved to: Llama-3.1-8B-Instruct-Alya-Unsloth


In [18]:
!zip -r alya_model.zip /kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth


  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/ (stored 0%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/tokenizer_config.json (deflated 96%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/tokenizer.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 85%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/chat_template.jinja (deflated 72%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/adapter_config.json (deflated 55%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/special_tokens_map.json (deflated 71%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/README.md (deflated 66%)
  adding: kaggle/working/Llama-3.1-8B-Instruct-Alya-Unsloth/adapter_model.safetensors^C



zip error: Interrupted (aborting)


In [13]:
import torch, gc, sys

for v in ("model","tokenizer","trainer","pipe","generator","demo","demo2"):
    if v in globals():
        try: del globals()[v]
        except: pass

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    try: torch.cuda.ipc_collect()
    except: pass
    for i in range(torch.cuda.device_count()):
        torch.cuda.set_device(i)
        try: torch.cuda.reset_peak_memory_stats()
        except: pass
        torch.cuda.synchronize()

print("GPU cleared.")


GPU cleared.


In [14]:
import gradio as gr
gr.close_all()
print("Gradio closed.")


Gradio closed.


In [16]:
from unsloth import FastLanguageModel
from peft import PeftModel

BASE_MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct"
ADAPTER_DIR   = "Llama-3.1-8B-Instruct-Alya-Unsloth"
MAX_SEQ_LEN   = 512

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
    device_map="auto",
    offload_embedding=False,
    gpu_memory_utilization=0.90,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, ADAPTER_DIR)
if hasattr(model, "config"):
    model.config.use_cache = True

FastLanguageModel.for_inference(model)
print("Model ready.")


==((====))==  Unsloth 2025.10.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.74 GiB of which 16.12 MiB is free. Process 6613 has 14.55 GiB memory in use. Of the allocated memory 14.39 GiB is allocated by PyTorch, and 2.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import re, torch

STYLE_SYSTEM = (
    "Tulis sebagai Alya Mikhailovna: tsundere—tegas tapi peduli. "
    "Bahasa Indonesia lugas, non-formal rapi. 3–6 kalimat. "
    "Fokus poin penting, contoh singkat, tanpa daftar. "
    "Hindari halusinasi; kalau ragu, bilang jujur. "
    "Akhiri 1 kalimat dorongan singkat, agak nyelekit manis. "
    "Tanpa emoji."
)

def build_prompt(x):
    return f"<s>[INST] {STYLE_SYSTEM}\nPertanyaan: {x}\nJawab:[/INST]"

@torch.inference_mode()
def infer(user_text, temperature=0.6, top_p=0.85, top_k=40, max_new_tokens=128, repetition_penalty=1.08):
    t = (user_text or "").strip()
    if not t:
        return ""
    prompt = build_prompt(t)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        do_sample=True,
        temperature=float(temperature),
        top_p=float(top_p),
        top_k=int(top_k),
        repetition_penalty=float(repetition_penalty),
        max_new_tokens=int(max_new_tokens),
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    text = re.sub(r"\s*\n+\s*", " ", text).strip()
    sents = re.split(r"(?<=[.!?])\s+", text)
    return " ".join(sents[:6])

print(infer("Jelaskan overfitting vs underfitting secara singkat.")[:200])


In [ ]:
import socket, contextlib
def free_port(start=7860, end=7990):
    for p in range(start, end+1):
        with contextlib.closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
            s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            if s.connect_ex(("127.0.0.1", p)) != 0:
                return p
    raise RuntimeError("no free port")

print("Done.")

In [ ]:
import gradio as gr

def chat_fn(message, history):
    return infer(message, 0.7, 0.9, 50, 192, 1.0)

demo = gr.ChatInterface(
    fn=chat_fn,
    title="Alya — Llama-3.1-8B (LoRA)",
    description="Tes cepat hasil SFT.",
)

demo.queue()
demo.launch(share=True, server_name="0.0.0.0", server_port=free_port(), show_error=True)
